In [1]:
import re
from nltk.tokenize import TweetTokenizer
import pandas as pd
from ast import literal_eval
import numpy as np
import os

!pip install transformers
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
df_all_tweets = pd.read_csv('/content/drive/My Drive/DATA SCIENCE MSC/DSM500 - Final Project/Datasets/all_tweets.csv', index_col=0)

## Full Classification Example
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest?text=Covid+cases+are+increasing+fast%21

In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)

def preprocess_single_text(text):
  new_text = []
  for t in text.split(" "):
      t = '@user' if t.startswith('@') and len(t) > 1 else t
      t = 'http' if t.startswith('http') else t
      new_text.append(t)
  return " ".join(new_text)

def preprocess_tweets(df):
  df['tweet_text'] = df['tweet_text'].map(lambda x: preprocess_single_text(x))
  return df


df_all_tweets = preprocess_tweets(df_all_tweets)

In [6]:
datasets_path = '/content/drive/My Drive/DATA SCIENCE MSC/DSM500 - Final Project/Datasets'

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

config = AutoConfig.from_pretrained(MODEL)

# PyTorch
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# TensorFlow
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def get_labels(scores):
  results = []
  for score in scores:
    # get softmax scores
    score = softmax(score)
    # get rankings
    ranking = np.argsort(score)[::-1]
    # get top softmax score
    top_score = score[ranking[0]]
    # get top label
    top_label = config.id2label[ranking[0]]
    # print(top_score, top_label)
    results.append({'label': top_label, 'score': top_score})
  return results


# labels = get_labels(scores)

In [7]:

split_dfs = np.array_split(df_all_tweets, 2000)
for i, df in enumerate(split_dfs):
  split_dfs[i] = [i,df]


tried just doing it tweet by tweet but didn't go well very slow

In [8]:
def get_sentiment_2(dfs, start=0, end=2000):
  # split df into chunks
    
  for info in dfs[start:end]:
    # preprocess tweets
    # df = preprocess_tweets(df)
    print(info[0])
  
    # convert to list
    processed_tweets = info[1]['tweet_text'].to_list()
    # encode inputs PYTORCH
    # encoded_inputs = tokenizer(processed_tweets, padding=True, truncation=True, max_length = 512, return_tensors='pt')
    # get outputs
    # outputs = model(**encoded_inputs)
    # get scores
    # scores = outputs[0].detach().numpy()

    # encode inputs TENSORFLOW
    encoded_inputs = tokenizer(processed_tweets, padding=True, truncation=True, max_length = 512, return_tensors='tf')
    # get outputs TENSORFLOW
    outputs = model(encoded_inputs)
    # get scores
    scores = outputs[0].numpy()
    # get labels
    labels = get_labels(scores)
    
    # create dataframe with sentiment
    sentiment_df = pd.DataFrame.from_dict(labels)
    # concat sentiment df with tweet df and return
    df_tweets_w_sentiment = pd.concat([df, sentiment_df], axis=1)
    # save to csv
    df_tweets_w_sentiment.to_csv(f'{datasets_path}/all_tweets_sentiment{info[0]}.csv')
   
  

10 sets of 5000 split in 4 min = 2000 min total  
3 sets of 1000 split in 450 secs = 2333 min
5 sets of 500 split in 30 min 3000 min = min (capped out ram)
So the bigger the chunks the slower the time overall

Try GPU implementation:
**bold text**3 sets of 1000 split - ram crash
10 sets of 5000 split - 10 secondst = 83 minute
10 sets of 2500 split - 20 seconds = 83
10 sets of 1500 split - 43 seconds = 107

Trying single iteration per tweet:
1 set of 1000 split in 250 secs = BAD

### error on partition 118 it breaks for some weird error

In [ ]:
# df_some_tweets = df_all_tweets.iloc[:1000].copy()
# start at 131
 
get_sentiment_2(split_dfs, 161)

In [21]:
# error on 118
df_118 = split_dfs[118][1]
df_119 = split_dfs[119][1]
# pd.DataFrame(split_dfs[118]).head()
# error on 131
# error on 156

In [22]:
df_118

,week,team,timestamp,tweet_text,userid
38114,12,raiders,2022-11-24 14:27:38+00:00,Happy Thanksgiving Raider Nation!!! 🔥🔥🔥🏴‍☠️⚔️☠...,930660272578146306
38115,12,raiders,2022-11-24 14:14:55+00:00,Fashion Stainless Steel Watches Men's 2021 New...,1574785943567011844
38116,12,raiders,2022-11-24 14:13:53+00:00,"@user Safe travels coach !! Let’s get the W, #...",26130756
38117,12,raiders,2022-11-24 14:04:34+00:00,"Happy Thanksgiving, #RaiderNation!",244306637
38118,12,raiders,2022-11-24 13:53:08+00:00,Seattle bound ☠️ #RaiderNation,884080825654026240
...,...,...,...,...,...
38432,12,raiders,2022-11-23 22:42:12+00:00,🎶 𝘐’𝘮𝘮𝘢 𝘵𝘢𝘬𝘦 𝘸𝘩𝘢𝘵’𝘴 𝘮𝘪𝘯𝘦 𝘴𝘰 𝘨𝘦𝘵 𝘰𝘶𝘵 𝘮𝘺 𝘸𝘢𝘺 𝘈𝘪𝘯...,1458673975110905856
38433,12,raiders,2022-11-23 22:40:06+00:00,AFC West Mix tape review: Looking ahead to res...,31194771
38434,12,raiders,2022-11-23 22:40:02+00:00,"#Raiders QB Derek Carr, on going uptempo: ""Pla...",47871775
38435,12,raiders,2022-11-23 22:35:31+00:00,Battling a torn rotator cuff and now dealing a...,537553144


In [23]:
df_119

,week,team,timestamp,tweet_text,userid
38437,12,raiders,2022-11-23 22:30:50+00:00,Despite a disappointing 3-7 start to the seaso...,15358759
38438,12,raiders,2022-11-23 22:30:09+00:00,Tune in tomorrow on Facebook Live bright and e...,965176734
38439,12,raiders,2022-11-23 22:23:29+00:00,Please don’t let him leave the Silver and Blac...,930660272578146306
38440,12,raiders,2022-11-23 22:19:35+00:00,Spoke with @ the O-line on their Thanksgiving ...,30169055
38441,12,raiders,2022-11-23 22:19:01+00:00,@user @user That’s what’s up!! Homegirl @user ...,3104985650
...,...,...,...,...,...
38755,12,rams,2022-11-27 15:11:00+00:00,"Aaron Donald, Bobby Wagner Among Leaders Stepp...",3037659434
38756,12,rams,2022-11-27 15:10:31+00:00,From @user The #Rams have no plans to shut QB ...,1555986762475966464
38757,12,rams,2022-11-27 15:08:13+00:00,@user Numbness that starts when he’s about to ...,1440488286644944906
38758,12,rams,2022-11-27 15:02:09+00:00,#Rams Matthew Stafford could miss the rest of ...,1387854072829599745
